<b>Multi-Directional Spread</b>
<n>Use 'trace-back' and 'back-fill' method where shortest distance from points across firefront back to previous day shared line is calculated.

1. Get smoothed interpolated daily perimeters 
2. Get shared fire-line Day1 & Day 2
3. Points along front line and shared line 
4. Find shortest distance from front line back to shared line 

For days > 1 then shared fireline can be past two days. 

In [1]:
import arcpy
from arcpy import env
from arcpy.sa import *
arcpy.overwriteoutput = True
import arcpy.management as DM
import arcpy.cartography as CA

env.workspace = "F:\DriversFireProject\TEMP"

import shutil
import itertools
import pandas as pd
from datetime import datetime, date, time, timedelta
import requests
import zipfile
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
# get list of files based on directory and extension inputs 
def shpFiles(rootPath, ext):
    emptyList = []
    root = rootPath
    for path, subdirs, files in os.walk(root):
        for names in files: 
            if names.endswith(ext) and not names.startswith("._"):
                emptyList.append(path + '\\' + names)
    return(emptyList)

# Create new folder in root path 
def createFolder(rootPath, folderName): 
    folderPath = os.path.join(rootPath, folderName) 
    if not os.path.exists(folderPath):
        os.makedirs(folderPath)
    return folderPath + "\\" 

In [3]:
import pandas as pd
from scipy.spatial.distance import cdist

def closest_point(point, points):
    """ Find closest point from a list of points. """
    return points[cdist([point], points).argmin()]

def match_value(df, col1, x, col2):
    """ Match value x from col1 row to value in col2. """
    return df[df[col1] == x][col2].values[0]

In [4]:
import math
from math import radians, degrees, sin, cos, asin, acos, sqrt
# def calculateDistance(pointA, pointB):
#     lat1 = math.radians(pointA[0])
#     lat2 = math.radians(pointB[0])
#     lon1 = math.radians(pointA[1])
#     lon2 = math.radians(pointB[1])
#     return 6371 * (
#         acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lon1 - lon2))
#     )

def calculate_initial_compass_bearing(pointA, pointB):
    """
    Calculates the bearing between two points.
    The formulae used is the following:
        θ = atan2(sin(Δlong).cos(lat2),
                  cos(lat1).sin(lat2) − sin(lat1).cos(lat2).cos(Δlong))
    :Parameters:
      - `pointA: The tuple representing the latitude/longitude for the
        first point. Latitude and longitude must be in decimal degrees
      - `pointB: The tuple representing the latitude/longitude for the
        second point. Latitude and longitude must be in decimal degrees
    :Returns:
      The bearing in degrees
    :Returns Type:
      float
    """
    if (type(pointA) != tuple) or (type(pointB) != tuple):
        raise TypeError("Only tuples are supported as arguments")

    lat1 = math.radians(pointA[0])
    lat2 = math.radians(pointB[0])

    diffLong = math.radians(pointB[1] - pointA[1])

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
            * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # Now we have the initial bearing but math.atan2 return values
    # from -180° to + 180° which is not what we want for a compass bearing
    # The solution is to normalize the initial bearing as shown below
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing

In [18]:
pointA = (41.01293623, -122.04922577)
pointB = (41.01176303, -122.03467066)
calculate_initial_compass_bearing(pointB, pointA)

276.1020670509138

In [5]:
interFiles = shpFiles(r'F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP', '.shp')

In [9]:
# delete days from simplified SHP if > 85% cumulative burned sum 
# first create DF with fires, year, day, area burned 
fire = []
year = [] 
day = []
area = [] 

for inter in interFiles: 
    print(inter)
    nm = inter.split("\\")
    shpName = nm[-1]
    nmm = shpName.split("_")
    fr = nmm[0]
    yr = nmm[1]
    if len(arcpy.ListFields(inter,"FireName"))== 0:  
        arcpy.AddField_management(inter, "FireName", "TEXT") 
    if len(arcpy.ListFields(inter,"Year"))== 0:  
        arcpy.AddField_management(inter, "Year", "TEXT") 
    if len(arcpy.ListFields(inter,"BurnedArea"))== 0:  
        arcpy.AddField_management(inter, "BurnedArea", "FLOAT") 
        arcpy.CalculateField_management(inter, "BurnedArea", "!SHAPE.AREA@HECTARES!", "PYTHON")
    with arcpy.da.UpdateCursor(inter, ['gridcode', 'BurnedArea', 'FireName', 'Year']) as cursor: 
        for row in cursor: 
            fire.append(fr)
            year.append(yr)
            day.append(row[0])
            area.append(row[1])
            row[2] = fr
            row[3] = str(yr)
            
            cursor.updateRow(row)

fireInfo = pd.DataFrame({'Fire': fire, 'Year':year, 'Day': day, 'Area':area})
fireInfo

F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2012\Bagleycomplex\Bagleycomplex_2012_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2012\Banner\Banner_2012_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2012\Chihuahua2\Chihuahua2_2012_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2012\Chips\Chips_2012_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2012\Dale\Dale_2012_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2012\Fortcomplex\Fortcomplex_2012_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2012\Fortcomplexgoff\Fortcomplexgoff_2012_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2012\George\George_2012_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2012\Highland\Highland_2012_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2012\Jawboneco

F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2014\Littledeer\Littledeer_2014_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2014\Lodgecomplex\Lodgecomplex_2014_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2014\Modocjulycomplexgulch\Modocjulycomplexgulch_2014_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2014\Monticello\Monticello_2014_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2014\Nicolls\Nicolls_2014_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2014\Sandiegocomplexcocos\Sandiegocomplexcocos_2014_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2014\Shirley\Shirley_2014_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2014\Tomahawk\Tomahawk_2014_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2014\Way\Way_2014_NAT.shp
F:\DriversFireProject\NaturalNeighborRe

F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2016\Sacata\Sacata_2016_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2016\Sangabrielcomplexfish\Sangabrielcomplexfish_2016_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2016\Sangabrielcomplexreservoir\Sangabrielcomplexreservoir_2016_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2016\Sawmill\Sawmill_2016_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2016\Sherpa\Sherpa_2016_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2016\Slate\Slate_2016_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2016\Soberanes\Soberanes_2016_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2016\Soupcomplexsoup2\Soupcomplexsoup2_2016_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2016\Trailhead\Trailhead_2016_NAT.shp
F:\DriversFireProject\Nat

F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2017\Winter\Winter_2017_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2017\Young\Young_2017_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2018\Alder\Alder_2018_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2018\Boot\Boot_2018_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2018\Camp\Camp_2018_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2018\Carr\Carr_2018_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2018\County\County_2018_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2018\Cranston\Cranston_2018_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2018\Delta\Delta_2018_NAT.shp
F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2018\Donnell\Donnell_2018_NAT.shp
F:\DriversFireProject\NaturalNeighborResul

,Fire,Year,Day,Area
0,Bagleycomplex,2012,232,542.397093
1,Bagleycomplex,2012,233,831.816076
2,Bagleycomplex,2012,234,1025.933322
3,Bagleycomplex,2012,235,1151.103974
4,Bagleycomplex,2012,236,1351.212020
...,...,...,...,...
1841,Tucker,2019,210,3467.872752
1842,Tucker,2019,211,9.607121
1843,Walker,2019,248,1465.060361
1844,Walker,2019,249,8942.161366


In [16]:
fireInfo.to_csv(r'F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\FireInfo_RAW.csv')

In [117]:
len(fireInfo['Fire'].unique())

286

In [8]:
fireCount = fireInfo.groupby(['Fire', 'Year'])
cumudf = fireCount.agg({'Day' : ['count'] , 'Area' : ['sum']}).reset_index()
cumudf.columns = ['Fire', 'Year', 'Original Total Days', 'Original Total Area']
newDF = fireInfo.merge(cumudf, on=['Fire', 'Year'])
newDF['Ratio'] = newDF['Area']  / newDF['Original Total Area'] 
newDF['cumsum'] = newDF.groupby(['Fire', 'Year'])['Ratio'].cumsum()
#newDF = newDF[newDF['cumsum'] < .90]   
newDF = newDF[(newDF['Original Total Days'] < 6) | (newDF['cumsum'] < .80)]
newCount = newDF.groupby(['Fire', 'Year'])
newCount = newCount.agg({'Day' : ['count'] , 'Area' : ['sum']}).reset_index()
newCount.columns = ['Fire', 'Year', 'Filtered Total Days', 'Filtered Total Area']
new = newDF.merge(newCount, on=['Fire', 'Year'])
new.to_csv(r'F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\FireInfo_Filtered.csv')

In [10]:
# filter out if days dont match days in new df 
# create new filtered_simplified folder 
for inter in interFiles: 
    print(inter)
    nm = inter.split("\\")
    shpName = nm[-1]
    nmm = shpName.split("_")
    fr = nmm[0]
    yr = nmm[1]
    with arcpy.da.UpdateCursor(inter, ['gridcode']) as cursor: 
        for row in cursor: 
            day = row[0]
            result = (new.Fire == fr) & (new.Year == str(yr)) & (new.Day == int(day))
            if not any(result) == True: 
                print('deleted: ' + str(day))
                cursor.deleteRow()
            else: 
                continue 


F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\2012\Bagleycomplex\Bagleycomplex_2012_NAT.shp
deleted: 241
deleted: 242
deleted: 243
deleted: 244
deleted: 245
deleted: 246
deleted: 247
deleted: 248
deleted: 249


In [5]:
FinalDF = pd.read_csv(r'F:\DriversFireProject\NaturalNeighborResults\FinalDF.csv', index_col=0)
FinalDF = FinalDF[~FinalDF['Fire'].isin(['Colby', 'Cherokee'])]
sample = FinalDF.groupby(['Fire', 'Year']).size().reset_index()
fire = sample['Fire'].tolist()
FIRES = sample['Fire'].tolist()
YEARS =  sample['Year'].tolist()

In [7]:
len(FIRES)

255

In [5]:
#"Bagleycomplex", "American", "Hirz", "Carr", "Mendocinocomplexranch", "County",
#2012, 2013, 2018, 2018, 2018, 2018,
FIRES = [ "Mendocinocomplexriver", "Ferguson", "Donnell", "Tubbs", "Atlas", "Nuns", "Happycampcomplexfryingpan"]
YEARS = [ 2018, 2018, 2018, 2017, 2017, 2017, 2014]

In [22]:
def firstDayVectors(firstDay):
    SQL1 = "gridcode = {}".format(firstDay)
    FirstDayPoly = arcpy.MakeFeatureLayer_management(simp, DayPath + fr + "_" + str(yr) + str(firstDay) + "_temp.shp", SQL1)
    poly = arcpy.EliminatePolygonPart_management(FirstDayPoly, DayPath + str(firstDay) +  "_tempfilt.shp", 
                                                             "PERCENT","", 3, "ANY")
    polyCursor = arcpy.da.SearchCursor(poly, ["SHAPE@"])
    polyInfo = polyCursor.next()[0]

    name = fr  + "_" +  str(yr) + "_VII_M6.shp"
    ignition = os.path.join(ignitionPath, name)
    
    pointCursor = arcpy.da.SearchCursor(ignition, ["SHAPE@X", "SHAPE@Y"])
    ignitionCoord = pointCursor.next()
    ignitionX = ignitionCoord[0]
    ignitionY = ignitionCoord[1]
    
    ignitionPoint = arcpy.PointGeometry(arcpy.Point(ignitionX, ignitionY))
    
    ansA = polyInfo.contains(ignitionPoint)
    
    if ansA == False: 
        ignition = arcpy.FeatureToPoint_management(poly, os.path.join(ignitionFoldr, name) , "INSIDE")
    else: 
        ignition = arcpy.CopyFeatures_management(ignition, os.path.join(ignitionFoldr, name)) 
        
    DayLINE = arcpy.MakeFeatureLayer_management(poly, DayPath + fr + "_" + str(yr) + "_" + str(firstDay) +"_dayLine.shp")
    DayPoints = arcpy.GeneratePointsAlongLines_management(DayLINE, DayPath + fr + "_" + str(yr) + "_" + str(firstDay) +"_DayPoints.shp", 
                                                                  'DISTANCE', Distance= 0.0008)
    
    arcpy.AddField_management(ignition, "StoreID", "LONG")  
    arcpy.CalculateField_management(ignition, "StoreID", '!FID!')
    
    nearTable = arcpy.arcpy.GenerateNearTable_analysis(DayPoints, ignition, CSVPath + fr + "_" + str(yr) + "_" +str(firstDay) +"_NearTable.csv",
                                                               "", "LOCATION", "ANGLE", "CLOSEST")
    out_feature_class = os.path.join(nearTablePath, fr + "_" + str(yr) + "_" + str(firstDay) +"_NearTable.shp")
    nearTableSHP = arcpy.management.XYTableToPoint(nearTable, 
                                            out_feature_class, 'FROM_X', 'FROM_Y')
    lines = arcpy.ba.DesireLines(nearTableSHP, ignition, DayPath + fr + "_" + str(yr) + "_" + str(firstDay) +"_tempVectors.shp",
                                 "NEAR_FID", "StoreID", "STRAIGHT_LINE_DISTANCE", "KILOMETERS", None, "AWAY_FROM_STORES")
    lines_Clipped = arcpy.Clip_analysis(lines, poly, VectorLinesPath + fr + "_" + str(yr) + "_" + str(firstDay) +"_Vectors.shp")
    
    arcpy.AddField_management(lines_Clipped, "Fire", "TEXT")  
    arcpy.AddField_management(lines_Clipped, "Year", "TEXT")  
    arcpy.AddField_management(lines_Clipped, "JulianDay", "TEXT")
    arcpy.AddField_management(lines_Clipped, "Direction", "FLOAT")  
    arcpy.AddField_management(lines_Clipped, "GroupByDir", "SHORT")  
    with arcpy.da.UpdateCursor(lines_Clipped, ['Fire', 'Year', 'JulianDay', 'Direction', 'NEAR_X', 'NEAR_Y' , 'FROM_X', 'FROM_Y', 'GroupByDir']) as cursor:
         for row in cursor:
            row[0] = fr
            row[1] = str(yr)
            row[2] = str(firstDay)
            row[3] = calculate_initial_compass_bearing(tuple((row[5], row[4])), tuple((row[7], row[6])))
            if 0 <= row[3] < 22.5 or 337.5 < row[3] <= 360:  # NORTH
                row[8] = 1
            elif 22.5 <= row[3] < 67.5: # NORTHEAST
                row[8] = 2
            elif 67.5 <= row[3] < 112.5: # EAST 
                row[8] = 3
            elif 112.5 <= row[3] < 157.5: #SOUTHEAST
                row[8] = 4
            elif 157.5 <= row[3] < 202.5: # SOUTH
                row[8] = 5
            elif 202.5 <= row[3] <= 247.5: # SOUTHWEST
                row[8] = 6
            elif 247.5 <= row[3] <= 292.5: # WEST
                row[8] = 7
            else: 
                row[8] = 8 # NORTHWEST
            cursor.updateRow(row)
    del row, cursor

    arcpy.TableToTable_conversion(lines_Clipped, CSVPath, fr + "_" + str(yr) + "_" + str(firstDay) + "_Final.csv")
    
    df = pd.read_csv(os.path.join(CSVPath, fr + "_" + str(yr) + "_" + str(firstDay) + "_Final.csv"), index_col=0)
    df = df[['Fire', 'Year', 'JulianDay', 'Direction', 'GroupByDir', 'Distance', 'NEAR_X', 'NEAR_Y', 'FROM_X', 'FROM_Y']]
    df.to_csv(os.path.join(CSVPath, fr + "_" + str(yr) + "_" + str(firstDay) + "_Final.csv"))
    

In [16]:
len(FIRES)

242

In [23]:
rootPath = r'F:\DriversFireProject'
fireInfo = pd.read_csv(r'F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\FireInfo_Filtered.csv', index_col=0)
inpath  = os.path.join(rootPath,'NaturalNeighborResults', 'Daily', 'SimplifiedSHP')
outpath = os.path.join(rootPath,'MultiSpread')
ignitionPath = r'F:\DriversFireProject\Ignition\Centroid'

for i, (fr, yr) in enumerate(zip(FIRES[182:], YEARS[182:])): 
    print(i, fr, yr)    
    yrPath = createFolder(outpath, str(yr))
    frPath = createFolder(yrPath, fr)
    tempPath = createFolder(frPath, "Temp")
    ignitionFoldr = createFolder(frPath, "Ignitions")
    CSVPath = createFolder(frPath,  "CSV")
    DayPath = createFolder(frPath,  "DayPoints")
    sharedPath = createFolder(frPath,  "SharedPoints")
    nearTablePath = createFolder(frPath,  "NearTable")
    VectorLinesPath = createFolder(frPath,  "VectorLines")
    MergedAllVector = createFolder(frPath,  "VectorLines_MergedAll")
    FinalCSVPath = createFolder(frPath,  "FinalCSV")
    
    simp = os.path.join(inpath, str(yr), fr, fr + "_" + str(yr) + '_NAT.shp')
    
    day = [] 
    field = "gridcode"
    cursor = arcpy.SearchCursor(simp)
    for row in cursor:
        day.append(row.getValue(field))
    
    if len(day) > 1: 
        day = sorted(day)
        dayPairs = list(zip(day, day[1:] + day[:1])) 
        dayPairs = dayPairs[:-1] 
        
        for i, pairs in enumerate(dayPairs): 
            try: 
                print(pairs)
                day1 = pairs[0]
                day2 = pairs[1]
                DayPath = createFolder(tempPath,  str(day2))
                DayPolygons = createFolder(tempPath, "DayPolygons")

                if i == 0: 
                    firstDayVectors(day1)
                    # Copy features for first 2 day  
                    SQL = "gridcode = {} Or gridcode = {}".format(day1,day2)
                    polyTemp = arcpy.MakeFeatureLayer_management(simp, DayPath + str(day1) + str(day2) +  "_daytemp.shp", SQL)
                    poly = arcpy.EliminatePolygonPart_management(polyTemp, DayPath + str(day1) + str(day2) +  "_tempfilt.shp", 
                                                                 "PERCENT","", 3, "ANY")
                    arcpy.CopyFeatures_management(poly, DayPolygons + "DayPolygon_tempfilt_" + str(i) + ".shp")
                    DayPerimeter = arcpy.MakeFeatureLayer_management(poly, DayPath + fr + "_" + str(yr) + "_DayPerim_" + str(day1) +".shp", 
                                                  "\"gridcode\" = {}".format(int(day2)))

                else: 

                    # get day 1 of previous pair 
                    prevDay = dayPairs[i-1][0]

                    # Copy features for first 2 day  
                    SQL1 = "gridcode = {}".format(day1)
                    PrevDayPoly = arcpy.MakeFeatureLayer_management(simp, DayPath + str(day1) + "_sharetemp.shp", SQL1)
                    # dissolve prev day and day 1 
                    pastDaysSHP = DayPolygons + "DayPolygon_tempfilt_" + str(i-1) + ".shp"

                    polymerged = arcpy.Merge_management([pastDaysSHP, PrevDayPoly], DayPolygons + "DayPolygon_tempfilt_" + str(i) + ".shp")

                    dissolved = arcpy.Dissolve_management(polymerged, DayPath + str(day1) + "_Distemp.shp",
                              ["FID"])

                    SQL2 = "gridcode = {}".format(day2)
                    Day2poly = arcpy.MakeFeatureLayer_management(simp, DayPath + str(day2) +  "_temp.shp", SQL2)
                    polyElim = arcpy.EliminatePolygonPart_management(Day2poly, DayPath + str(day1) + str(day2) +  "_tempfilt.shp", 
                                                                 "PERCENT","", 2, "ANY")

                    # merge with day 2
                    poly = arcpy.Merge_management([dissolved, polyElim], DayPath + str(day1) + str(day2) +  "_temp.shp")
        #                 poly = arcpy.EliminatePolygonPart_management(polyTemp, DayPath + str(day1) + str(day2) +  "_tempfilt.shp", 
        #                                                              "PERCENT","", 3, "ANY")
                    DayPerimeter = arcpy.MakeFeatureLayer_management(poly, DayPath + fr + "_" + str(yr) + "_DayPerim_" + str(day1) +".shp", 
                                                  "\"gridcode\" = {}".format(int(day2)))


                # 3. Convert to line 
                polyline = arcpy.PolygonToLine_management(poly, DayPath + fr + "_" + str(yr) +  "_shareLine_" + str(pairs[0]) + "_" + str(pairs[1]) +".shp")

                # 4. Extract only shared line where LEFT_FID != -1 
                sharedLine = arcpy.MakeFeatureLayer_management(polyline, DayPath + fr + "_" + str(yr) + "_Sharedtemp_" + str(pairs[1]) +".shp", 
                                                  "\"LEFT_FID\" > -1")

                arcpy.CopyFeatures_management(sharedLine, tempPath + fr + "_" + str(yr) +"_SharedLine_" + str(pairs[1]) +".shp")

                # if closed loop for shape-length < 0.01 Then delete for sharedline only 
        #             multishared = arcpy.MultipartToSinglepart_management(sharedLine,
        #                                        tempPath + fr + "_" + str(yr) +"_ShareLineMulti_" + str(pairs[1]) +".shp")

                arcpy.AddField_management(sharedLine, "Length", "DOUBLE") 
                arcpy.CalculateField_management(sharedLine, "Length", '!shape.length!')

                # snap the features really close together to make one closed loop 

                #arcpy.CalculateGeometryAttributes_management(multiDay, [["Shape_Length", "LENGTH_GEODESIC"]])

                rows = arcpy.da.UpdateCursor(sharedLine, ["Shape@", "Length"])  
                for rw in rows:  
                    geom = rw[0]
                    if geom.firstPoint.X == geom.lastPoint.X and rw[1] < 0.01:  
                        rows.deleteRow()
                del rw, rows  

                # 5. Extract firefront  where RIGHT_FID != max
                cursor = arcpy.da.SearchCursor(polyline, ("RIGHT_FID"))

                RFID = []
                for row in cursor:
                    RFID.append(row[0])
                del row, cursor

                DayLINE = arcpy.MakeFeatureLayer_management(polyline, DayPath + fr + "_" + str(yr) + "_" + str(pairs[1]) +"_dayLine.shp", 
                                                  "\"RIGHT_FID\" = {}".format(max(RFID)))

                arcpy.CopyFeatures_management(DayLINE, tempPath + fr + "_" + str(yr) + str(pairs[1]) +"_dayLine.shp") 

        #             # if closed loop for shape-length > 0.01 Then delete for dayline only 
        #             multiDay = arcpy.MultipartToSinglepart_management(DayLINE,
        #                                        tempPath + fr + "_" + str(yr) +"_DayLineMulti_" + str(pairs[1]) +".shp")

                arcpy.AddField_management(DayLINE, "Length", "DOUBLE") 
                arcpy.CalculateField_management(DayLINE, "Length", '!shape.length!')

                #arcpy.CalculateGeometryAttributes_management(multiDay, [["Shape_Length", "LENGTH_GEODESIC"]])

                rows = arcpy.da.UpdateCursor(DayLINE, ["Shape@", "Length"])  
                for rw in rows:  
                    geom = rw[0]
                    if geom.firstPoint.X == geom.lastPoint.X and rw[1] < 0.1:  
                        rows.deleteRow()
                del rw, rows 

                # 6. Create points along firefront  

                DayPoints = arcpy.GeneratePointsAlongLines_management(DayLINE, 
                                                                      DayPath + fr + "_" + str(yr) + "_" + str(pairs[1]) +"_DayPoints.shp", 
                                                                      'DISTANCE', Distance= 0.0008)

                # 7. Create points along shared line 
                sharedPoints = arcpy.GeneratePointsAlongLines_management(sharedLine, 
                                                                      sharedPath + fr + "_" + str(yr) + "_" + str(pairs[1]) +"_SharedPoints.shp", 
                                                                      'DISTANCE', Distance= 0.0008)

                # 8. Create NearTable 

                nearTable = arcpy.arcpy.GenerateNearTable_analysis(DayPoints, sharedPoints, nearTablePath + fr + "_" + str(yr) + "_" + str(pairs[1]) +"_NearTable.csv",
                                                                   "", "LOCATION", "ANGLE", "CLOSEST")

                arcpy.AddField_management(sharedPoints, "StoreID", "LONG")  
                arcpy.CalculateField_management(sharedPoints, "StoreID", '!FID!')
                arcpy.AddXY_management(sharedPoints)

                arcpy.TableToTable_conversion(sharedPoints, CSVPath, fr + "_" + str(yr) + "_" + str(pairs[1]) +"_sharedPoints.csv")
                sharedDF = pd.read_csv(os.path.join(CSVPath, fr + "_" + str(yr) + "_" + str(pairs[1]) +"_sharedPoints.csv"), index_col=0)
                nearDF = pd.read_csv(os.path.join(nearTablePath, fr + "_" + str(yr) + "_" + str(pairs[1]) +"_NearTable.csv"), index_col=0)

                nearDF = nearDF.sort_values(by=['NEAR_FID'])
                sharedDF = sharedDF.sort_values(by=['StoreID'])

                fromX = [] 
                fromY = [] 
                nearX = [] 
                nearY = [] 
                nearFID = [] 

                IDList = sharedDF['StoreID'].tolist()
                NearList = nearDF['NEAR_FID'].tolist()

                notInList = [x for x in IDList if x not in NearList]

                for idVAL in notInList:
                    i = sharedDF.StoreID[sharedDF.StoreID == idVAL].index[0]
                    shareX = sharedDF['POINT_X'][i]
                    shareY = sharedDF['POINT_Y'][i]
                    nearX.append(shareX)
                    nearY.append(shareY)
                    nearFID.append(sharedDF['StoreID'][i])

                    nearDF['point'] = [(x, y) for x,y in zip(nearDF['NEAR_X'], nearDF['NEAR_Y'])]
                    point = (shareX, shareY)
                    closePT = closest_point(point, list(nearDF['point']))

                    frmX = nearDF.loc[(nearDF['NEAR_X'] == closePT[0]) & (nearDF['NEAR_Y'] == closePT[1]), 'FROM_X'].iloc[0]
                    frmY = nearDF.loc[(nearDF['NEAR_X'] == closePT[0]) & (nearDF['NEAR_Y'] == closePT[1]), 'FROM_Y'].iloc[0]
                    fromX.append(frmX) 
                    fromY.append(frmY)

                frame1 = pd.DataFrame({'NEAR_FID': nearFID, 'NEAR_X': nearX,'NEAR_Y': nearY, 'FROM_X': fromX, 'FROM_Y': fromY})
                nearDF = nearDF[['NEAR_FID','NEAR_X', 'NEAR_Y', 'FROM_X', 'FROM_Y']]

                frames = [nearDF, frame1]
                result = pd.concat(frames)

                result['Fire'] = fr
                result['Year'] = str(yr)
                result['JulianDay'] = str(pairs[1])
                nearxy = list(zip(result['NEAR_Y'], result['NEAR_X'])) 
                fromxy = list(zip(result['FROM_Y'], result['FROM_X'])) 

                direction = []
                for xy in zip(nearxy, fromxy):
                    direction.append(calculate_initial_compass_bearing(xy[0], xy[1]))

                result['Direction'] = direction

                result.to_csv(CSVPath + fr + "_" + str(yr) + "_" + str(pairs[1]) +"_complete.csv")
                result = result.sort_values(by=['NEAR_FID'])
                out_feature_class = os.path.join(nearTablePath, fr + "_" + str(yr) + "_" + str(pairs[1]) +"_NearTable.shp")
                nearTableSHP = arcpy.management.XYTableToPoint(CSVPath + fr + "_" + str(yr) + "_" + str(pairs[1]) +"_complete.csv", 
                                                out_feature_class, 'FROM_X', 'FROM_Y')

                lines = arcpy.ba.DesireLines(nearTableSHP, sharedPoints, DayPath + fr + "_" + str(yr) + "_" + str(pairs[1]) +"_Vectors.shp",
                                     "NEAR_FID", "StoreID", "STRAIGHT_LINE_DISTANCE", "KILOMETERS", None, "AWAY_FROM_STORES")
                lines_Clipped = arcpy.Clip_analysis(lines, DayPerimeter, VectorLinesPath + fr + "_" + str(yr) + "_" + str(pairs[1]) +"_Vectors.shp")

                arcpy.AddField_management(lines_Clipped, "GroupByDir", "SHORT")  
                with arcpy.da.UpdateCursor(lines_Clipped, ['Direction','GroupByDir']) as cursor:
                     for row in cursor:
                        if 0 <= row[0] < 22.5 or 337.5 < row[0] <= 360:  # NORTH
                            row[1] = 1
                        elif 22.5 <= row[0] < 67.5: # NORTHEAST
                            row[1] = 2
                        elif 67.5 <= row[0] < 112.5: # EAST 
                            row[1] = 3
                        elif 112.5 <= row[0] < 157.5: #SOUTHEAST
                            row[1] = 4
                        elif 157.5 <= row[0] < 202.5: # SOUTH
                            row[1] = 5
                        elif 202.5 <= row[0] <= 247.5: # SOUTHWEST
                            row[1] = 6
                        elif 247.5 <= row[0] <= 292.5: # WEST
                            row[1] = 7
                        else: 
                            row[1] = 8 # NORTHWEST
                        cursor.updateRow(row)
                del row, cursor

                # to CSV 
                arcpy.TableToTable_conversion(lines_Clipped, CSVPath, fr + "_" + str(yr) + "_" + str(pairs[1]) +"_Final.csv")

                df = pd.read_csv(os.path.join(CSVPath, fr + "_" + str(yr) + "_" + str(pairs[1]) +"_Final.csv"), index_col=0)
                df = df[['Fire', 'Year', 'JulianDay', 'Direction', 'GroupByDir', 'Distance', 'NEAR_X', 'NEAR_Y', 'FROM_X', 'FROM_Y']]
                df.to_csv(os.path.join(CSVPath, fr + "_" + str(yr) + "_" + str(pairs[1]) +"_Final.csv"))

                shpList = shpFiles(VectorLinesPath, ".shp")
                if len(shpList) == len(dayPairs) + 1: 
                    merged = arcpy.Merge_management(shpList, os.path.join(MergedAllVector, fr + "_" + str(yr) + "_Vec.shp"))
                    arcpy.TableToTable_conversion(merged, FinalCSVPath, fr + "_" + str(yr) + "_FinalAll.csv")
                    finaldf = pd.read_csv(os.path.join(FinalCSVPath, fr + "_" + str(yr) + "_FinalAll.csv"), index_col=0)
                    finaldf = finaldf.merge(fireInfo, how = "left", left_on=['Fire', 'Year', 'JulianDay'], right_on=['Fire', 'Year', 'Day'])
                    finaldf = finaldf[['Fire', 'Year', 'JulianDay', 'Area', 'Direction', 'GroupByDir', 'Distance', 'NEAR_X', 'NEAR_Y', 'FROM_X', 'FROM_Y']]
                    finaldf.to_csv(os.path.join(FinalCSVPath, fr + "_" + str(yr) + "_FinalAll.csv"))
            except: 
                print("error: ", pairs)
                continue
    elif len(day) == 1: 
        Cursor = arcpy.da.SearchCursor(simp,["gridcode"])
        day1 = Cursor.next()[0]
        firstDayVectors(day1)
                

0 Routecomplexbuck 2015
(214, 215)


SystemError: <built-in function isinstance> returned a result with an error set

SystemError: <built-in function isinstance> returned a result with an error set

SystemError: <built-in function isinstance> returned a result with an error set

(215, 216)
(216, 217)
(217, 218)
1 Routecomplexjohnson 2015
(212, 213)
(213, 214)
(214, 215)
(215, 216)
(216, 217)
(217, 218)
(218, 219)
(219, 220)
(220, 221)
(221, 222)
(222, 223)
(223, 224)
2 Ruthcomplexdutchman 2017
3 Ruthcomplexwillie 2017
(223, 224)
(224, 225)
(225, 226)
(226, 227)
4 Rye 2017
5 Sacata 2016
(285, 286)
6 Salmonaugustcomplexwallow 2017
(226, 227)
error:  (226, 227)
(227, 228)
(228, 229)
(229, 230)
(230, 231)
(231, 232)
(232, 233)
(233, 234)
(234, 235)
(235, 236)
(236, 237)
(237, 238)
(238, 239)
(239, 240)
(240, 241)
(241, 242)
(242, 243)
(243, 244)
7 Salmonrivercomplexboulder 2013
(213, 214)
(214, 215)
(215, 216)
(216, 217)
(217, 218)
(218, 219)
(219, 220)
(220, 221)
(221, 222)
(222, 223)
8 Salmonrivercomplexshelly 2013
(213, 214)
(214, 215)
(215, 216)
(216, 217)
9 Sand 2019
10 Sandiegocomplexcocos 2014
(134, 135)
11 Sangabrielcomplexfish 2016
(172, 173)
12 Sangabrielcomplexreservoir 2016
13 Sawmill 2016
14 Scotts 2012
(251, 252)
(252, 253)
15 Sherpa 2016
(167, 168)


In [13]:
# for fires with only one day copy files into VectorLines_MergedALL folder and add csv into FinalCSV folder 
# for fires with error, if folder empty and vectorfolder len > 1 then merge into folder and add CSV 

rootPath = r'F:\DriversFireProject'
fireInfo = pd.read_csv(r'F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\FireInfo_Filtered.csv', index_col=0)
inpath  = os.path.join(rootPath,'NaturalNeighborResults', 'Daily', 'SimplifiedSHP')
outpath = os.path.join(rootPath,'MultiSpread')
ignitionPath = r'F:\DriversFireProject\Ignition\Centroid'

for i, (fr, yr) in enumerate(zip(FIRES, YEARS)): 
    yrPath = createFolder(outpath, str(yr))
    frPath = createFolder(yrPath, fr)
    VectorLinesPath = createFolder(frPath, "VectorLines")
    MergedAllVector = createFolder(frPath, "VectorLines_MergedAll")
    FinalCSVPath = createFolder(frPath, "FinalCSV")
    if len(os.listdir(MergedAllVector)) == 0:
        print(fr, yr)
        try: 
            shpList = shpFiles(VectorLinesPath, '.shp')
            if len(shpList) == 1:
                # copy features management 
                copy = arcpy.CopyFeatures_management(shpList[0], os.path.join(MergedAllVector, fr + "_" + str(yr) + "_Vec.shp"))
                arcpy.TableToTable_conversion(copy, FinalCSVPath, fr + "_" + str(yr) + "_FinalAll.csv")
                finaldf = pd.read_csv(os.path.join(FinalCSVPath, fr + "_" + str(yr) + "_FinalAll.csv"), index_col=0)
                finaldf = finaldf.merge(fireInfo, how = "left", left_on=['Fire', 'Year', 'JulianDay'], right_on=['Fire', 'Year', 'Day'])
                finaldf = finaldf[['Fire', 'Year', 'JulianDay', 'Area', 'Direction', 'GroupByDir', 'Distance', 'NEAR_X', 'NEAR_Y', 'FROM_X', 'FROM_Y']]
                # add csv 
                finaldf.to_csv(os.path.join(FinalCSVPath, fr + "_" + str(yr) + "_FinalAll.csv"))

            elif len(shpList) > 1:
                # merge features management 
                merged = arcpy.Merge_management(shpList, os.path.join(MergedAllVector, fr + "_" + str(yr) + "_Vec.shp"))
                arcpy.TableToTable_conversion(merged, FinalCSVPath, fr + "_" + str(yr) + "_FinalAll.csv")
                finaldf = pd.read_csv(os.path.join(FinalCSVPath, fr + "_" + str(yr) + "_FinalAll.csv"), index_col=0)
                finaldf = finaldf.merge(fireInfo, how = "left", left_on=['Fire', 'Year', 'JulianDay'], right_on=['Fire', 'Year', 'Day'])
                finaldf = finaldf[['Fire', 'Year', 'JulianDay', 'Area', 'Direction', 'GroupByDir', 'Distance', 'NEAR_X', 'NEAR_Y', 'FROM_X', 'FROM_Y']]
                # add csv 
                finaldf.to_csv(os.path.join(FinalCSVPath, fr + "_" + str(yr) + "_FinalAll.csv"))
            else: 
                print(fr, yr, 'none')
        except: 
            print(fr, yr, 'error')
            continue


Creeklac 2017
Curry 2016
Dale 2012
Dale 2012 error
Deer 2016
Deluz 2013
Etiwanda 2014
Falls 2013
Falls 2013 none
Forkcomplexblue 2015
Forkcomplexpeak 2015
Forkcomplexrail 2015
Front 2018
Front 2018 error
Gate 2017
General 2013
George 2012
Goose 2016
Grand 2013
Gulchshu 2014
Hat 2018
Highland 2012
Holcomb 2017
Hudson 2017
Jerusalem 2015
Lumpkiin 2015
Lumpkin 2015
Manzanita 2017
Marshes 2016
Mdflakecomplex 2012
Modocjulycomplexlake 2017
North 2018
Orleanscomplexukonom 2017
Park 2017
Parker2 2017
Pass 2012
Prescott 2017
R1ranch 2019
Reservoir 2016
Ruthcomplexdutchman 2017
Rye 2017
Salmonaugustcomplexwallow 2017
Sand 2019
Sangabrielcomplexreservoir 2016
Sawmill 2016
Sawmill 2016 error
Shockey 2012
Shockey 2012 error
Shucreek 2018
Sitescomplex 2012
Solimar 2015
Sulfur 2017
Tassajarra 2015
Tesla 2015
Tomahawk 2014
Turkey 2012
Valley 2018
Wall 2017
Way 2014
White 2013
Willard 2016
Winter 2017


In [7]:
rootPath = r'F:\DriversFireProject'
fireInfo = pd.read_csv(r'F:\DriversFireProject\NaturalNeighborResults\Daily\SimplifiedSHP\FireInfo_Filtered.csv', index_col=0)
inpath  = os.path.join(rootPath,'NaturalNeighborResults', 'Daily', 'SimplifiedSHP')
outpath = os.path.join(rootPath,'MultiSpread')

for i, (fr, yr) in enumerate(zip(FIRES, YEARS)): 
    yrPath = createFolder(outpath, str(yr))
    frPath = createFolder(yrPath, fr)
    VectorLinesPath = createFolder(frPath, "VectorLines")
    MergedAllVector = createFolder(frPath, "VectorLines_MergedAll")
    FinalCSVPath = createFolder(frPath, "FinalCSV")
    if len(os.listdir(FinalCSVPath)) == 0:
        print(fr, yr)
        try:
            copy = os.path.join(MergedAllVector, fr + "_" + str(yr) + "_Vec.shp")
            arcpy.TableToTable_conversion(copy, FinalCSVPath, fr + "_" + str(yr) + "_FinalAll.csv")
            finaldf = pd.read_csv(os.path.join(FinalCSVPath, fr + "_" + str(yr) + "_FinalAll.csv"), index_col=0)
            finaldf = finaldf.merge(fireInfo, how = "left", left_on=['Fire', 'Year', 'JulianDay'], right_on=['Fire', 'Year', 'Day'])
            finaldf = finaldf[['Fire', 'Year', 'JulianDay', 'Area', 'Direction', 'GroupByDir', 'Distance', 'NEAR_X', 'NEAR_Y', 'FROM_X', 'FROM_Y']]
            # add csv 
            finaldf.to_csv(os.path.join(FinalCSVPath, fr + "_" + str(yr) + "_FinalAll.csv"))
        except: 
            continue


Falls 2013
